<a href="https://colab.research.google.com/github/lorettarehm/AIML/blob/main/LR_Capstone_SalesForecasting_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Sales Forecasting with Time Series Analysis

## Project Overview
This notebook provides a comprehensive analysis and forecasting solution for restaurant sales data using advanced time series techniques. The project demonstrates:

- **Objective**: Predict future sales using historical transaction data
- **Scope**: Multi-restaurant, multi-item sales forecasting with seasonal patterns
- **Methods**: Traditional ML, ARIMA/SARIMA, and Prophet models
- **Outcome**: Actionable insights for business planning and inventory management

### Key Features
- ✅ Dynamic path handling for portability across environments
- ✅ Comprehensive exploratory data analysis with visualizations
- ✅ Advanced preprocessing with reusable functions
- ✅ Time-based feature engineering
- ✅ Multiple forecasting models comparison
- ✅ Interactive visualizations
- ✅ Business insights and recommendations

## 1. Environment Setup and Data Loading

### Import Required Libraries
Setting up the environment with all necessary libraries for data analysis, visualization, and modeling.

In [ ]:
# Core data manipulation libraries
import pandas as pd
import numpy as np
import os
import warnings
from pathlib import Path

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Time series libraries
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from prophet import Prophet

# Date and time handling
from datetime import datetime, timedelta
import dateutil.parser

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set random seeds for reproducibility
np.random.seed(42)

print("✅ All libraries imported successfully!")

### Dynamic Path Configuration

This section implements portable path handling that works across different environments (local, Colab, Jupyter, etc.)

In [ ]:
def setup_data_paths():
    """
    Setup data paths dynamically based on the environment.
    Works in Google Colab, local Jupyter, and other environments.
    """
    # Check if running in Google Colab
    try:
        import google.colab
        # Running in Google Colab
        datasets_path = Path('/content/drive/MyDrive/AIML/CAPSTONE/datasets')
        if not datasets_path.exists():
            # Fallback to current directory if drive not mounted
            datasets_path = Path('./datasets')
        print(f"🔗 Google Colab environment detected")
    except ImportError:
        # Running locally or in other environments
        datasets_path = Path('./datasets')
        print(f"💻 Local environment detected")
    
    # Create datasets directory if it doesn't exist
    datasets_path.mkdir(exist_ok=True)
    
    print(f"📁 Data path: {datasets_path.absolute()}")
    return datasets_path

# Setup paths
datasets_path = setup_data_paths()

# Define file paths
files = {
    'sales': datasets_path / 'sales.csv',
    'items': datasets_path / 'items.csv',
    'restaurants': datasets_path / 'restaurants.csv'
}

print(f"\n📋 Expected files:")
for name, path in files.items():
    exists = "✅" if path.exists() else "❌"
    print(f"  {exists} {name}: {path.name}")

### Data Loading and Initial Inspection

Loading the three main datasets and performing initial data quality checks.

In [ ]:
def load_datasets(files):
    """
    Load all datasets with error handling and initial validation.
    """
    datasets = {}
    
    for name, file_path in files.items():
        try:
            if file_path.exists():
                df = pd.read_csv(file_path)
                datasets[name] = df
                print(f"✅ Loaded {name}: {df.shape[0]:,} rows, {df.shape[1]} columns")
            else:
                print(f"❌ File not found: {file_path}")
                # Create sample data if files don't exist
                print(f"🔧 Creating sample {name} data...")
                if name == 'sales':
                    datasets[name] = create_sample_sales_data()
                elif name == 'items':
                    datasets[name] = create_sample_items_data()
                elif name == 'restaurants':
                    datasets[name] = create_sample_restaurants_data()
        except Exception as e:
            print(f"❌ Error loading {name}: {e}")
    
    return datasets

def create_sample_sales_data():
    """Create sample sales data if file doesn't exist"""
    dates = pd.date_range('2020-01-01', '2021-12-31', freq='D')
    np.random.seed(42)
    
    data = []
    for i in range(10000):  # Reduced sample size
        data.append({
            'date': np.random.choice(dates).strftime('%Y-%m-%d'),
            'item_id': np.random.randint(1, 51),
            'store_id': np.random.randint(1, 6),
            'item_count': np.random.choice([1, 2, 3, 4], p=[0.6, 0.25, 0.1, 0.05])
        })
    return pd.DataFrame(data)

def create_sample_items_data():
    """Create sample items data if file doesn't exist"""
    categories = ['Main Course', 'Appetizer', 'Dessert', 'Beverage']
    data = []
    for i in range(1, 51):
        data.append({
            'id': i,
            'name': f'Item {i}',
            'category': np.random.choice(categories),
            'price': round(np.random.uniform(5.99, 29.99), 2),
            'kcal': np.random.randint(50, 800)
        })
    return pd.DataFrame(data)

def create_sample_restaurants_data():
    """Create sample restaurants data if file doesn't exist"""
    return pd.DataFrame({
        'id': [1, 2, 3, 4, 5],
        'name': ['Restaurant A', 'Restaurant B', 'Restaurant C', 'Restaurant D', 'Restaurant E'],
        'city': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'],
        'state': ['NY', 'CA', 'IL', 'TX', 'AZ']
    })

# Load datasets
datasets = load_datasets(files)
sales_df = datasets['sales']
items_df = datasets['items']
restaurants_df = datasets['restaurants']

print(f"\n📊 Dataset Summary:")
print(f"  Sales transactions: {len(sales_df):,}")
print(f"  Unique items: {len(items_df):,}")
print(f"  Restaurant locations: {len(restaurants_df):,}")

## 2. Data Quality Assessment and Preprocessing

### Initial Data Quality Check
Comprehensive assessment of data quality, missing values, and data types.

In [ ]:
def analyze_data_quality(df, name):
    """
    Comprehensive data quality analysis function.
    """
    print(f"\n{'='*50}")
    print(f"📋 DATA QUALITY REPORT: {name.upper()}")
    print(f"{'='*50}")
    
    # Basic info
    print(f"\n📏 Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    
    # Data types
    print(f"\n🏷️  Data Types:")
    for col, dtype in df.dtypes.items():
        print(f"   {col}: {dtype}")
    
    # Missing values
    missing = df.isnull().sum()
    print(f"\n❓ Missing Values:")
    if missing.sum() == 0:
        print(f"   ✅ No missing values found!")
    else:
        for col, count in missing[missing > 0].items():
            pct = (count / len(df)) * 100
            print(f"   {col}: {count:,} ({pct:.1f}%)")
    
    # Duplicates
    duplicates = df.duplicated().sum()
    print(f"\n🔄 Duplicate Rows: {duplicates:,}")
    
    # Preview
    print(f"\n👀 Sample Data:")
    display(df.head())
    
    return df

# Analyze each dataset
sales_df = analyze_data_quality(sales_df, "Sales")
items_df = analyze_data_quality(items_df, "Items")
restaurants_df = analyze_data_quality(restaurants_df, "Restaurants")

### Data Preprocessing Functions

Reusable functions for data cleaning, type conversion, and preprocessing.

In [ ]:
def preprocess_sales_data(sales_df):
    """
    Comprehensive preprocessing for sales data.
    """
    print("🔧 Preprocessing sales data...")
    
    # Make a copy to avoid modifying original
    df = sales_df.copy()
    
    # Convert date column to datetime
    df['date'] = pd.to_datetime(df['date'])
    print(f"   ✅ Converted date column to datetime")
    
    # Sort by date
    df = df.sort_values('date').reset_index(drop=True)
    print(f"   ✅ Sorted by date")
    
    # Handle missing values
    initial_rows = len(df)
    df = df.dropna()
    if initial_rows != len(df):
        print(f"   ✅ Removed {initial_rows - len(df)} rows with missing values")
    
    # Remove duplicates
    initial_rows = len(df)
    df = df.drop_duplicates()
    if initial_rows != len(df):
        print(f"   ✅ Removed {initial_rows - len(df)} duplicate rows")
    
    # Validate data ranges
    df = df[df['item_count'] > 0]  # Remove zero or negative counts
    print(f"   ✅ Validated data ranges")
    
    print(f"   📊 Final shape: {df.shape}")
    return df

def detect_outliers(df, column, method='iqr'):
    """
    Detect outliers using IQR or Z-score method.
    """
    if method == 'iqr':
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    else:  # z-score
        z_scores = np.abs((df[column] - df[column].mean()) / df[column].std())
        outliers = df[z_scores > 3]
    
    return outliers

def handle_outliers(df, column, method='cap'):
    """
    Handle outliers by capping or removing.
    """
    outliers_before = len(detect_outliers(df, column))
    
    if method == 'cap':
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
    elif method == 'remove':
        outliers = detect_outliers(df, column)
        df = df.drop(outliers.index)
    
    outliers_after = len(detect_outliers(df, column))
    print(f"   📊 {column}: {outliers_before} → {outliers_after} outliers")
    
    return df

# Apply preprocessing
sales_clean = preprocess_sales_data(sales_df)

print(f"\n✨ Preprocessing complete!")
print(f"   Original sales data: {len(sales_df):,} rows")
print(f"   Cleaned sales data: {len(sales_clean):,} rows")
print(f"   Date range: {sales_clean['date'].min()} to {sales_clean['date'].max()}")

### Data Integration

Merging datasets to create a comprehensive view for analysis.

In [ ]:
def merge_datasets(sales_df, items_df, restaurants_df):
    """
    Merge all datasets into a comprehensive dataframe.
    """
    print("🔗 Merging datasets...")
    
    # Merge sales with items
    merged = sales_df.merge(items_df, left_on='item_id', right_on='id', how='left', suffixes=('', '_item'))
    print(f"   ✅ Merged with items: {len(merged):,} rows")
    
    # Merge with restaurants
    merged = merged.merge(restaurants_df, left_on='store_id', right_on='id', how='left', suffixes=('', '_restaurant'))
    print(f"   ✅ Merged with restaurants: {len(merged):,} rows")
    
    # Calculate total sales amount
    merged['total_sales'] = merged['item_count'] * merged['price']
    print(f"   ✅ Calculated total sales")
    
    # Clean up duplicate columns
    columns_to_drop = ['id_item', 'id_restaurant']
    merged = merged.drop(columns=[col for col in columns_to_drop if col in merged.columns])
    
    # Rename columns for clarity
    merged = merged.rename(columns={
        'name': 'item_name',
        'name_restaurant': 'restaurant_name'
    })
    
    print(f"   📊 Final merged dataset: {merged.shape}")
    print(f"   💰 Total sales value: ${merged['total_sales'].sum():,.2f}")
    
    return merged

# Merge datasets
merged_df = merge_datasets(sales_clean, items_df, restaurants_df)

# Display sample of merged data
print(f"\n👀 Sample of merged data:")
display(merged_df.head())

print(f"\n📋 Column summary:")
for col in merged_df.columns:
    print(f"   {col}: {merged_df[col].dtype}")

## 3. Comprehensive Exploratory Data Analysis (EDA)

### Sales Distribution and Trends Analysis
Analyzing patterns in sales data across time, categories, and locations.

In [ ]:
def create_sales_overview_plots(df):
    """
    Create comprehensive overview plots for sales data.
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Sales Data Overview', fontsize=16, fontweight='bold')
    
    # 1. Daily sales trend
    daily_sales = df.groupby('date')['total_sales'].sum().reset_index()
    axes[0, 0].plot(daily_sales['date'], daily_sales['total_sales'], color='steelblue', alpha=0.7)
    axes[0, 0].set_title('Daily Sales Trend', fontweight='bold')
    axes[0, 0].set_xlabel('Date')
    axes[0, 0].set_ylabel('Total Sales ($)')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Sales by category
    category_sales = df.groupby('category')['total_sales'].sum().sort_values(ascending=True)
    axes[0, 1].barh(category_sales.index, category_sales.values, color='lightcoral')
    axes[0, 1].set_title('Sales by Item Category', fontweight='bold')
    axes[0, 1].set_xlabel('Total Sales ($)')
    
    # 3. Sales by restaurant
    restaurant_sales = df.groupby('restaurant_name')['total_sales'].sum().sort_values(ascending=True)
    axes[1, 0].barh(restaurant_sales.index, restaurant_sales.values, color='lightgreen')
    axes[1, 0].set_title('Sales by Restaurant', fontweight='bold')
    axes[1, 0].set_xlabel('Total Sales ($)')
    
    # 4. Item count distribution
    axes[1, 1].hist(df['item_count'], bins=20, color='gold', alpha=0.7, edgecolor='black')
    axes[1, 1].set_title('Distribution of Item Count per Transaction', fontweight='bold')
    axes[1, 1].set_xlabel('Item Count')
    axes[1, 1].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()

# Create overview plots
create_sales_overview_plots(merged_df)

# Print key statistics
print("📊 KEY STATISTICS")
print(f"{'='*40}")
print(f"Total Sales Revenue: ${merged_df['total_sales'].sum():,.2f}")
print(f"Average Transaction Value: ${merged_df['total_sales'].mean():.2f}")
print(f"Median Transaction Value: ${merged_df['total_sales'].median():.2f}")
print(f"Number of Transactions: {len(merged_df):,}")
print(f"Date Range: {merged_df['date'].min().strftime('%Y-%m-%d')} to {merged_df['date'].max().strftime('%Y-%m-%d')}")
print(f"Unique Items Sold: {merged_df['item_id'].nunique()}")
print(f"Active Restaurants: {merged_df['store_id'].nunique()}")

### Time Series Analysis

Deep dive into temporal patterns and seasonality in sales data.

In [ ]:
def analyze_temporal_patterns(df):
    """
    Analyze temporal patterns in sales data.
    """
    # Create time-based features
    df_temp = df.copy()
    df_temp['year'] = df_temp['date'].dt.year
    df_temp['month'] = df_temp['date'].dt.month
    df_temp['day_of_week'] = df_temp['date'].dt.dayofweek
    df_temp['day_of_year'] = df_temp['date'].dt.dayofyear
    df_temp['week_of_year'] = df_temp['date'].dt.isocalendar().week
    df_temp['is_weekend'] = df_temp['day_of_week'].isin([5, 6])
    
    # Day names for better visualization
    day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Temporal Pattern Analysis', fontsize=16, fontweight='bold')
    
    # 1. Monthly sales pattern
    monthly_sales = df_temp.groupby('month')['total_sales'].sum()
    axes[0, 0].bar(monthly_sales.index, monthly_sales.values, color='skyblue')
    axes[0, 0].set_title('Sales by Month', fontweight='bold')
    axes[0, 0].set_xlabel('Month')
    axes[0, 0].set_ylabel('Total Sales ($)')
    axes[0, 0].set_xticks(range(1, 13))
    axes[0, 0].set_xticklabels(month_names, rotation=45)
    
    # 2. Day of week pattern
    dow_sales = df_temp.groupby('day_of_week')['total_sales'].sum()
    axes[0, 1].bar(dow_sales.index, dow_sales.values, color='lightcoral')
    axes[0, 1].set_title('Sales by Day of Week', fontweight='bold')
    axes[0, 1].set_xlabel('Day of Week')
    axes[0, 1].set_ylabel('Total Sales ($)')
    axes[0, 1].set_xticks(range(7))
    axes[0, 1].set_xticklabels(day_names, rotation=45)
    
    # 3. Weekend vs Weekday
    weekend_sales = df_temp.groupby('is_weekend')['total_sales'].sum()
    weekend_labels = ['Weekday', 'Weekend']
    axes[0, 2].pie(weekend_sales.values, labels=weekend_labels, autopct='%1.1f%%', 
                   colors=['lightblue', 'lightgreen'])
    axes[0, 2].set_title('Weekend vs Weekday Sales', fontweight='bold')
    
    # 4. Weekly rolling average
    daily_sales = df_temp.groupby('date')['total_sales'].sum().reset_index()
    daily_sales['rolling_7d'] = daily_sales['total_sales'].rolling(window=7).mean()
    axes[1, 0].plot(daily_sales['date'], daily_sales['total_sales'], alpha=0.3, color='gray', label='Daily')
    axes[1, 0].plot(daily_sales['date'], daily_sales['rolling_7d'], color='red', linewidth=2, label='7-day average')
    axes[1, 0].set_title('Sales Trend with 7-Day Moving Average', fontweight='bold')
    axes[1, 0].set_xlabel('Date')
    axes[1, 0].set_ylabel('Total Sales ($)')
    axes[1, 0].legend()
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 5. Hourly pattern (if time data available, simulate it)
    # Simulate hourly patterns for demonstration
    np.random.seed(42)
    hours = range(24)
    hourly_pattern = [0.2, 0.1, 0.05, 0.05, 0.1, 0.3, 0.8, 1.2, 1.0, 0.9, 
                     1.1, 1.5, 1.8, 1.6, 1.4, 1.2, 1.0, 1.1, 1.3, 1.4, 1.2, 0.8, 0.5, 0.3]
    axes[1, 1].plot(hours, hourly_pattern, marker='o', color='orange', linewidth=2)
    axes[1, 1].set_title('Simulated Hourly Sales Pattern', fontweight='bold')
    axes[1, 1].set_xlabel('Hour of Day')
    axes[1, 1].set_ylabel('Relative Sales Volume')
    axes[1, 1].grid(True, alpha=0.3)
    
    # 6. Seasonal decomposition preview
    if len(daily_sales) > 365:  # Need enough data for seasonal decomposition
        # Simple trend analysis
        axes[1, 2].scatter(daily_sales.index, daily_sales['total_sales'], alpha=0.5, color='purple')
        # Fit trend line
        z = np.polyfit(daily_sales.index, daily_sales['total_sales'], 1)
        p = np.poly1d(z)
        axes[1, 2].plot(daily_sales.index, p(daily_sales.index), "r--", alpha=0.8, linewidth=2)
        axes[1, 2].set_title('Sales Trend Analysis', fontweight='bold')
        axes[1, 2].set_xlabel('Days from Start')
        axes[1, 2].set_ylabel('Total Sales ($)')
    else:
        axes[1, 2].text(0.5, 0.5, 'Insufficient data\nfor trend analysis', 
                       ha='center', va='center', transform=axes[1, 2].transAxes)
        axes[1, 2].set_title('Trend Analysis (Insufficient Data)', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    return df_temp

# Analyze temporal patterns
df_with_time_features = analyze_temporal_patterns(merged_df)

# Statistical summary of temporal patterns
print("\n📈 TEMPORAL PATTERN INSIGHTS")
print(f"{'='*50}")

# Weekend vs weekday analysis
weekend_avg = df_with_time_features[df_with_time_features['is_weekend']]['total_sales'].mean()
weekday_avg = df_with_time_features[~df_with_time_features['is_weekend']]['total_sales'].mean()
print(f"Average Weekend Transaction: ${weekend_avg:.2f}")
print(f"Average Weekday Transaction: ${weekday_avg:.2f}")
print(f"Weekend Premium: {((weekend_avg / weekday_avg) - 1) * 100:.1f}%")

# Seasonal analysis
monthly_avg = df_with_time_features.groupby('month')['total_sales'].mean()
best_month = monthly_avg.idxmax()
worst_month = monthly_avg.idxmin()
print(f"\nBest performing month: {best_month} (${monthly_avg[best_month]:.2f} avg)")
print(f"Worst performing month: {worst_month} (${monthly_avg[worst_month]:.2f} avg)")

### Correlation Analysis

Understanding relationships between different variables using correlation heatmaps and analysis.

In [ ]:
def create_correlation_analysis(df):
    """
    Create comprehensive correlation analysis with heatmaps.
    """
    # Prepare data for correlation analysis
    df_corr = df.copy()
    
    # Create encoded categorical variables for correlation
    df_corr['category_encoded'] = pd.Categorical(df_corr['category']).codes
    df_corr['restaurant_encoded'] = pd.Categorical(df_corr['restaurant_name']).codes
    df_corr['state_encoded'] = pd.Categorical(df_corr['state']).codes
    
    # Select numerical columns for correlation
    numerical_cols = ['item_count', 'price', 'kcal', 'total_sales', 
                     'year', 'month', 'day_of_week', 'day_of_year',
                     'category_encoded', 'restaurant_encoded', 'state_encoded']
    
    # Filter columns that exist in the dataframe
    available_cols = [col for col in numerical_cols if col in df_corr.columns]
    
    corr_matrix = df_corr[available_cols].corr()
    
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    
    # 1. Full correlation heatmap
    sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, 
                square=True, ax=axes[0], fmt='.2f', cbar_kws={'shrink': 0.8})
    axes[0].set_title('Feature Correlation Heatmap', fontweight='bold', pad=20)
    axes[0].tick_params(axis='both', rotation=45)
    
    # 2. Sales-focused correlation
    sales_corr = corr_matrix['total_sales'].sort_values(ascending=False)
    colors = ['green' if x > 0 else 'red' for x in sales_corr.values]
    axes[1].barh(range(len(sales_corr)), sales_corr.values, color=colors, alpha=0.7)
    axes[1].set_yticks(range(len(sales_corr)))
    axes[1].set_yticklabels(sales_corr.index)
    axes[1].set_xlabel('Correlation with Total Sales')
    axes[1].set_title('Features Correlation with Sales', fontweight='bold', pad=20)
    axes[1].axvline(x=0, color='black', linestyle='-', alpha=0.3)
    
    # Add correlation values as text
    for i, v in enumerate(sales_corr.values):
        axes[1].text(v + (0.02 if v >= 0 else -0.02), i, f'{v:.2f}', 
                    va='center', ha='left' if v >= 0 else 'right')
    
    plt.tight_layout()
    plt.show()
    
    return corr_matrix, sales_corr

# Create correlation analysis
corr_matrix, sales_correlations = create_correlation_analysis(df_with_time_features)

print("\n🔗 CORRELATION INSIGHTS")
print(f"{'='*40}")
print("Top positive correlations with sales:")
positive_corr = sales_correlations[sales_correlations > 0].sort_values(ascending=False)
for feature, corr in positive_corr.head(5).items():
    if feature != 'total_sales':  # Exclude self-correlation
        print(f"  {feature}: {corr:.3f}")

print("\nTop negative correlations with sales:")
negative_corr = sales_correlations[sales_correlations < 0].sort_values()
for feature, corr in negative_corr.head(3).items():
    print(f"  {feature}: {corr:.3f}")

## 4. Advanced Feature Engineering

### Time-Based Feature Engineering
Creating sophisticated time-based features for improved forecasting accuracy.

In [ ]:
def create_advanced_time_features(df):
    """
    Create comprehensive time-based features for forecasting.
    """
    print("🔧 Creating advanced time-based features...")
    
    df_features = df.copy()
    
    # Basic time features
    df_features['year'] = df_features['date'].dt.year
    df_features['month'] = df_features['date'].dt.month
    df_features['day'] = df_features['date'].dt.day
    df_features['day_of_week'] = df_features['date'].dt.dayofweek
    df_features['day_of_year'] = df_features['date'].dt.dayofyear
    df_features['week_of_year'] = df_features['date'].dt.isocalendar().week
    df_features['quarter'] = df_features['date'].dt.quarter
    
    # Advanced time features
    df_features['is_weekend'] = df_features['day_of_week'].isin([5, 6]).astype(int)
    df_features['is_month_start'] = df_features['date'].dt.is_month_start.astype(int)
    df_features['is_month_end'] = df_features['date'].dt.is_month_end.astype(int)
    df_features['is_quarter_start'] = df_features['date'].dt.is_quarter_start.astype(int)
    df_features['is_quarter_end'] = df_features['date'].dt.is_quarter_end.astype(int)
    
    # Cyclical features (sin/cos encoding for cyclical nature)
    df_features['month_sin'] = np.sin(2 * np.pi * df_features['month'] / 12)
    df_features['month_cos'] = np.cos(2 * np.pi * df_features['month'] / 12)
    df_features['day_of_week_sin'] = np.sin(2 * np.pi * df_features['day_of_week'] / 7)
    df_features['day_of_week_cos'] = np.cos(2 * np.pi * df_features['day_of_week'] / 7)
    df_features['day_of_year_sin'] = np.sin(2 * np.pi * df_features['day_of_year'] / 365)
    df_features['day_of_year_cos'] = np.cos(2 * np.pi * df_features['day_of_year'] / 365)
    
    # Holiday features (simplified)
    df_features['is_holiday'] = 0
    # New Year's period
    df_features.loc[(df_features['month'] == 1) & (df_features['day'] <= 5), 'is_holiday'] = 1
    # Christmas period
    df_features.loc[(df_features['month'] == 12) & (df_features['day'] >= 20), 'is_holiday'] = 1
    
    print(f"   ✅ Created {len([col for col in df_features.columns if col not in df.columns])} new time features")
    
    return df_features

def create_lag_features(df, target_col='total_sales', lags=[1, 3, 7, 14, 30]):
    """
    Create lag features for time series forecasting.
    """
    print(f"🔧 Creating lag features for {target_col}...")
    
    # Aggregate daily sales first
    daily_sales = df.groupby('date')[target_col].sum().reset_index()
    daily_sales = daily_sales.sort_values('date')
    
    # Create lag features
    for lag in lags:
        daily_sales[f'{target_col}_lag_{lag}'] = daily_sales[target_col].shift(lag)
    
    # Rolling window features
    for window in [3, 7, 14, 30]:
        daily_sales[f'{target_col}_rolling_mean_{window}'] = daily_sales[target_col].rolling(window=window).mean()
        daily_sales[f'{target_col}_rolling_std_{window}'] = daily_sales[target_col].rolling(window=window).std()
        daily_sales[f'{target_col}_rolling_min_{window}'] = daily_sales[target_col].rolling(window=window).min()
        daily_sales[f'{target_col}_rolling_max_{window}'] = daily_sales[target_col].rolling(window=window).max()
    
    # Exponential moving averages
    for alpha in [0.1, 0.3, 0.5]:
        daily_sales[f'{target_col}_ema_{alpha}'] = daily_sales[target_col].ewm(alpha=alpha).mean()
    
    print(f"   ✅ Created lag and rolling features")
    
    return daily_sales

def create_aggregated_features(df):
    """
    Create aggregated features for different time windows.
    """
    print("🔧 Creating aggregated features...")
    
    # Weekly aggregations
    df['week_start'] = df['date'] - pd.to_timedelta(df['date'].dt.dayofweek, unit='d')
    weekly_agg = df.groupby(['week_start', 'store_id']).agg({
        'total_sales': ['sum', 'mean', 'count'],
        'item_count': ['sum', 'mean'],
        'price': 'mean'
    }).reset_index()
    
    # Flatten column names
    weekly_agg.columns = ['_'.join(col).strip() if col[1] else col[0] for col in weekly_agg.columns.values]
    weekly_agg = weekly_agg.rename(columns={'week_start_': 'week_start', 'store_id_': 'store_id'})
    
    # Monthly aggregations
    df['month_start'] = df['date'].dt.to_period('M').dt.start_time
    monthly_agg = df.groupby(['month_start', 'store_id']).agg({
        'total_sales': ['sum', 'mean', 'count'],
        'item_count': ['sum', 'mean'],
        'price': 'mean'
    }).reset_index()
    
    # Flatten column names
    monthly_agg.columns = ['_'.join(col).strip() if col[1] else col[0] for col in monthly_agg.columns.values]
    monthly_agg = monthly_agg.rename(columns={'month_start_': 'month_start', 'store_id_': 'store_id'})
    
    print(f"   ✅ Created weekly aggregations: {weekly_agg.shape}")
    print(f"   ✅ Created monthly aggregations: {monthly_agg.shape}")
    
    return weekly_agg, monthly_agg

# Apply feature engineering
df_with_features = create_advanced_time_features(df_with_time_features)
daily_sales_with_lags = create_lag_features(df_with_features)
weekly_agg, monthly_agg = create_aggregated_features(df_with_features)

print(f"\n✨ Feature Engineering Summary:")
print(f"   Original features: {len(df_with_time_features.columns)}")
print(f"   Enhanced features: {len(df_with_features.columns)}")
print(f"   Daily sales with lags: {daily_sales_with_lags.shape}")
print(f"   Weekly aggregations: {weekly_agg.shape}")
print(f"   Monthly aggregations: {monthly_agg.shape}")

# Display sample of enhanced features
print(f"\n👀 Sample of time-enhanced features:")
feature_cols = ['date', 'total_sales', 'month_sin', 'month_cos', 'day_of_week_sin', 
               'is_weekend', 'is_holiday', 'quarter']
available_features = [col for col in feature_cols if col in df_with_features.columns]
display(df_with_features[available_features].head())

## 5. Machine Learning Models

### Traditional Machine Learning Approach
Implementing baseline models with cross-validation and performance evaluation.

In [ ]:
def prepare_ml_data(daily_sales_df, target_col='total_sales'):
    """
    Prepare data for machine learning models.
    """
    print("🔧 Preparing data for ML models...")
    
    # Remove rows with NaN values (from lag features)
    ml_data = daily_sales_df.dropna().copy()
    
    # Feature columns (exclude target and date)
    feature_cols = [col for col in ml_data.columns if col not in ['date', target_col]]
    
    X = ml_data[feature_cols]
    y = ml_data[target_col]
    
    # Time-based split (last 20% for testing)
    split_index = int(len(ml_data) * 0.8)
    
    X_train = X.iloc[:split_index]
    X_test = X.iloc[split_index:]
    y_train = y.iloc[:split_index]
    y_test = y.iloc[split_index:]
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"   ✅ Training set: {X_train.shape}")
    print(f"   ✅ Test set: {X_test.shape}")
    print(f"   ✅ Features: {len(feature_cols)}")
    
    return X_train_scaled, X_test_scaled, y_train, y_test, scaler, feature_cols, ml_data

def evaluate_model(y_true, y_pred, model_name):
    """
    Comprehensive model evaluation.
    """
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # MAPE (Mean Absolute Percentage Error)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\n📊 {model_name} Performance:")
    print(f"   RMSE: ${rmse:,.2f}")
    print(f"   MAE:  ${mae:,.2f}")
    print(f"   R²:   {r2:.3f}")
    print(f"   MAPE: {mape:.2f}%")
    
    return {'rmse': rmse, 'mae': mae, 'r2': r2, 'mape': mape}

def train_ml_models(X_train, X_test, y_train, y_test):
    """
    Train and evaluate multiple ML models.
    """
    print("🚀 Training machine learning models...")
    
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    }
    
    results = {}
    trained_models = {}
    
    for name, model in models.items():
        print(f"\n🔄 Training {name}...")
        
        # Train model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate
        results[name] = evaluate_model(y_test, y_pred, name)
        results[name]['predictions'] = y_pred
        trained_models[name] = model
    
    return results, trained_models

# Prepare data and train models
if len(daily_sales_with_lags) > 50:  # Ensure we have enough data
    X_train, X_test, y_train, y_test, scaler, feature_cols, ml_data = prepare_ml_data(daily_sales_with_lags)
    ml_results, trained_models = train_ml_models(X_train, X_test, y_train, y_test)
    
    # Create comparison plot
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Model comparison
    metrics = ['rmse', 'mae', 'r2', 'mape']
    model_names = list(ml_results.keys())
    
    metric_data = {metric: [ml_results[model][metric] for model in model_names] for metric in metrics}
    
    # RMSE comparison
    axes[0].bar(model_names, [ml_results[model]['rmse'] for model in model_names], 
               color=['skyblue', 'lightcoral'])
    axes[0].set_title('Model Comparison - RMSE', fontweight='bold')
    axes[0].set_ylabel('RMSE ($)')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Prediction vs Actual for best model
    best_model = min(ml_results.keys(), key=lambda x: ml_results[x]['rmse'])
    y_pred_best = ml_results[best_model]['predictions']
    
    axes[1].scatter(y_test, y_pred_best, alpha=0.6, color='green')
    axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    axes[1].set_xlabel('Actual Sales ($)')
    axes[1].set_ylabel('Predicted Sales ($)')
    axes[1].set_title(f'Predictions vs Actual - {best_model}', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n🏆 Best Model: {best_model} (RMSE: ${ml_results[best_model]['rmse']:,.2f})")
    
else:
    print("⚠️ Insufficient data for ML model training")
    ml_results = {}
    trained_models = {}

## 6. Advanced Time Series Forecasting

### Time Series Stationarity and Preparation
Analyzing stationarity and preparing data for ARIMA/SARIMA models.

In [ ]:
def check_stationarity(timeseries, title):
    """
    Check stationarity using Augmented Dickey-Fuller test.
    """
    print(f"\n📊 Stationarity Test: {title}")
    print('=' * 50)
    
    # Perform Augmented Dickey-Fuller test
    result = adfuller(timeseries, autolag='AIC')
    
    print(f'ADF Statistic: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value:.3f}')
    
    if result[1] <= 0.05:
        print("✅ Series is stationary (reject null hypothesis)")
        is_stationary = True
    else:
        print("❌ Series is non-stationary (fail to reject null hypothesis)")
        is_stationary = False
    
    return is_stationary

def prepare_timeseries_data(df, target_col='total_sales'):
    """
    Prepare time series data for ARIMA/SARIMA modeling.
    """
    print("🔧 Preparing time series data...")
    
    # Create daily time series
    daily_ts = df.groupby('date')[target_col].sum().reset_index()
    daily_ts = daily_ts.set_index('date').sort_index()
    
    # Fill missing dates with interpolation
    daily_ts = daily_ts.asfreq('D')
    daily_ts[target_col] = daily_ts[target_col].interpolate(method='linear')
    
    print(f"   ✅ Time series shape: {daily_ts.shape}")
    print(f"   ✅ Date range: {daily_ts.index.min()} to {daily_ts.index.max()}")
    
    return daily_ts

def seasonal_decomposition_analysis(ts, period=30):
    """
    Perform seasonal decomposition analysis.
    """
    print(f"\n🔍 Seasonal Decomposition Analysis (period={period})")
    
    if len(ts) >= 2 * period:
        decomposition = seasonal_decompose(ts.squeeze(), model='additive', period=period)
        
        fig, axes = plt.subplots(4, 1, figsize=(15, 12))
        
        decomposition.observed.plot(ax=axes[0], title='Original Time Series', color='blue')
        decomposition.trend.plot(ax=axes[1], title='Trend Component', color='green')
        decomposition.seasonal.plot(ax=axes[2], title='Seasonal Component', color='orange')
        decomposition.resid.plot(ax=axes[3], title='Residual Component', color='red')
        
        for ax in axes:
            ax.tick_params(axis='x', rotation=45)
            ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        return decomposition
    else:
        print(f"   ⚠️ Insufficient data for seasonal decomposition (need at least {2*period} points)")
        return None

# Prepare time series data
if len(df_with_features) > 0:
    ts_data = prepare_timeseries_data(df_with_features)
    
    # Check stationarity
    is_stationary = check_stationarity(ts_data['total_sales'].dropna(), "Original Series")
    
    # Seasonal decomposition
    decomposition = seasonal_decomposition_analysis(ts_data['total_sales'].dropna())
    
    # If non-stationary, try differencing
    if not is_stationary:
        ts_data['total_sales_diff'] = ts_data['total_sales'].diff()
        is_stationary_diff = check_stationarity(ts_data['total_sales_diff'].dropna(), "First Difference")
        
        if not is_stationary_diff:
            ts_data['total_sales_diff2'] = ts_data['total_sales_diff'].diff()
            check_stationarity(ts_data['total_sales_diff2'].dropna(), "Second Difference")
    
else:
    print("⚠️ No data available for time series analysis")
    ts_data = None

### ARIMA and SARIMA Models

Implementing traditional statistical time series models.

In [ ]:
def fit_arima_model(ts, order=(1, 1, 1)):
    """
    Fit ARIMA model to time series data.
    """
    print(f"\n🔄 Fitting ARIMA{order} model...")
    
    try:
        # Split data for training and testing
        train_size = int(len(ts) * 0.8)
        train, test = ts[:train_size], ts[train_size:]
        
        # Fit ARIMA model
        model = ARIMA(train, order=order)
        fitted_model = model.fit()
        
        # Make predictions
        forecast = fitted_model.forecast(steps=len(test))
        
        # Calculate metrics
        mse = mean_squared_error(test, forecast)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(test, forecast)
        
        print(f"   ✅ ARIMA{order} - RMSE: ${rmse:.2f}, MAE: ${mae:.2f}")
        print(f"   📊 AIC: {fitted_model.aic:.2f}, BIC: {fitted_model.bic:.2f}")
        
        return {
            'model': fitted_model,
            'forecast': forecast,
            'train': train,
            'test': test,
            'rmse': rmse,
            'mae': mae,
            'aic': fitted_model.aic,
            'bic': fitted_model.bic
        }
        
    except Exception as e:
        print(f"   ❌ Error fitting ARIMA{order}: {e}")
        return None

def fit_sarima_model(ts, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12)):
    """
    Fit SARIMA model to time series data.
    """
    print(f"\n🔄 Fitting SARIMA{order}x{seasonal_order} model...")
    
    try:
        # Split data for training and testing
        train_size = int(len(ts) * 0.8)
        train, test = ts[:train_size], ts[train_size:]
        
        # Fit SARIMA model
        model = SARIMAX(train, order=order, seasonal_order=seasonal_order)
        fitted_model = model.fit(disp=False)
        
        # Make predictions
        forecast = fitted_model.forecast(steps=len(test))
        
        # Calculate metrics
        mse = mean_squared_error(test, forecast)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(test, forecast)
        
        print(f"   ✅ SARIMA{order}x{seasonal_order} - RMSE: ${rmse:.2f}, MAE: ${mae:.2f}")
        print(f"   📊 AIC: {fitted_model.aic:.2f}, BIC: {fitted_model.bic:.2f}")
        
        return {
            'model': fitted_model,
            'forecast': forecast,
            'train': train,
            'test': test,
            'rmse': rmse,
            'mae': mae,
            'aic': fitted_model.aic,
            'bic': fitted_model.bic
        }
        
    except Exception as e:
        print(f"   ❌ Error fitting SARIMA{order}x{seasonal_order}: {e}")
        return None

# Fit ARIMA and SARIMA models
arima_sarima_results = {}

if ts_data is not None and len(ts_data['total_sales'].dropna()) > 50:
    ts_clean = ts_data['total_sales'].dropna()
    
    # Try different ARIMA configurations
    arima_configs = [(1, 1, 1), (2, 1, 1), (1, 1, 2), (2, 1, 2)]
    
    for order in arima_configs:
        result = fit_arima_model(ts_clean, order)
        if result:
            arima_sarima_results[f'ARIMA{order}'] = result
    
    # Try SARIMA with different configurations
    sarima_configs = [
        ((1, 1, 1), (1, 1, 1, 7)),   # Weekly seasonality
        ((1, 1, 1), (1, 1, 1, 30)),  # Monthly seasonality
    ]
    
    for order, seasonal_order in sarima_configs:
        if len(ts_clean) > 2 * seasonal_order[3]:  # Need enough data for seasonality
            result = fit_sarima_model(ts_clean, order, seasonal_order)
            if result:
                arima_sarima_results[f'SARIMA{order}x{seasonal_order}'] = result
    
    # Find best model
    if arima_sarima_results:
        best_model_name = min(arima_sarima_results.keys(), 
                             key=lambda x: arima_sarima_results[x]['rmse'])
        best_result = arima_sarima_results[best_model_name]
        
        print(f"\n🏆 Best ARIMA/SARIMA Model: {best_model_name}")
        print(f"   RMSE: ${best_result['rmse']:.2f}")
        print(f"   MAE: ${best_result['mae']:.2f}")
        print(f"   AIC: {best_result['aic']:.2f}")
        
        # Plot best model results
        plt.figure(figsize=(14, 6))
        
        # Plot training data
        plt.plot(best_result['train'].index, best_result['train'].values, 
                label='Training Data', color='blue', alpha=0.7)
        
        # Plot test data
        plt.plot(best_result['test'].index, best_result['test'].values, 
                label='Actual Test Data', color='green')
        
        # Plot forecast
        plt.plot(best_result['test'].index, best_result['forecast'], 
                label=f'{best_model_name} Forecast', color='red', linestyle='--')
        
        plt.title(f'{best_model_name} - Sales Forecasting', fontsize=14, fontweight='bold')
        plt.xlabel('Date')
        plt.ylabel('Total Sales ($)')
        plt.legend()
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
else:
    print("⚠️ Insufficient data for ARIMA/SARIMA modeling")

### Prophet Model

Implementing Facebook Prophet for robust time series forecasting with automatic seasonality detection.

In [ ]:
def fit_prophet_model(df, target_col='total_sales'):
    """
    Fit Prophet model to time series data.
    """
    print("\n🔮 Fitting Prophet model...")
    
    try:
        # Prepare data for Prophet (requires 'ds' and 'y' columns)
        daily_sales = df.groupby('date')[target_col].sum().reset_index()
        daily_sales.columns = ['ds', 'y']
        
        # Split data
        train_size = int(len(daily_sales) * 0.8)
        train_data = daily_sales[:train_size]
        test_data = daily_sales[train_size:]
        
        # Initialize and fit Prophet model
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            seasonality_mode='additive',
            changepoint_prior_scale=0.05
        )
        
        # Add custom seasonalities
        model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
        
        # Fit the model
        model.fit(train_data)
        
        # Create future dataframe for predictions
        future = model.make_future_dataframe(periods=len(test_data))
        
        # Make predictions
        forecast = model.predict(future)
        
        # Extract test predictions
        test_forecast = forecast.tail(len(test_data))
        
        # Calculate metrics
        mse = mean_squared_error(test_data['y'], test_forecast['yhat'])
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(test_data['y'], test_forecast['yhat'])
        r2 = r2_score(test_data['y'], test_forecast['yhat'])
        
        print(f"   ✅ Prophet Model Performance:")
        print(f"      RMSE: ${rmse:.2f}")
        print(f"      MAE:  ${mae:.2f}")
        print(f"      R²:   {r2:.3f}")
        
        return {
            'model': model,
            'forecast': forecast,
            'train_data': train_data,
            'test_data': test_data,
            'test_forecast': test_forecast,
            'rmse': rmse,
            'mae': mae,
            'r2': r2
        }
        
    except Exception as e:
        print(f"   ❌ Error fitting Prophet model: {e}")
        return None

def plot_prophet_results(prophet_result):
    """
    Create comprehensive Prophet results visualization.
    """
    if prophet_result is None:
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Prophet Model Analysis', fontsize=16, fontweight='bold')
    
    # 1. Forecast plot
    forecast = prophet_result['forecast']
    train_data = prophet_result['train_data']
    test_data = prophet_result['test_data']
    
    axes[0, 0].plot(train_data['ds'], train_data['y'], 'b-', label='Training Data', alpha=0.7)
    axes[0, 0].plot(test_data['ds'], test_data['y'], 'g-', label='Actual Test Data', linewidth=2)
    axes[0, 0].plot(forecast['ds'], forecast['yhat'], 'r--', label='Prophet Forecast', alpha=0.8)
    axes[0, 0].fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], 
                           color='red', alpha=0.2, label='Confidence Interval')
    axes[0, 0].set_title('Sales Forecast with Prophet', fontweight='bold')
    axes[0, 0].set_xlabel('Date')
    axes[0, 0].set_ylabel('Total Sales ($)')
    axes[0, 0].legend()
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Residuals plot
    test_forecast = prophet_result['test_forecast']
    residuals = test_data['y'].values - test_forecast['yhat'].values
    axes[0, 1].scatter(test_forecast['yhat'], residuals, alpha=0.6)
    axes[0, 1].axhline(y=0, color='red', linestyle='--')
    axes[0, 1].set_title('Residuals Plot', fontweight='bold')
    axes[0, 1].set_xlabel('Predicted Values')
    axes[0, 1].set_ylabel('Residuals')
    
    # 3. Weekly seasonality
    weekly_seasonality = forecast[['ds', 'weekly']].copy()
    weekly_seasonality['day_of_week'] = weekly_seasonality['ds'].dt.dayofweek
    weekly_avg = weekly_seasonality.groupby('day_of_week')['weekly'].mean()
    day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    axes[1, 0].bar(range(7), weekly_avg.values, color='skyblue')
    axes[1, 0].set_title('Weekly Seasonality', fontweight='bold')
    axes[1, 0].set_xlabel('Day of Week')
    axes[1, 0].set_ylabel('Effect on Sales')
    axes[1, 0].set_xticks(range(7))
    axes[1, 0].set_xticklabels(day_names)
    
    # 4. Yearly trend
    axes[1, 1].plot(forecast['ds'], forecast['trend'], color='green', linewidth=2)
    axes[1, 1].set_title('Trend Component', fontweight='bold')
    axes[1, 1].set_xlabel('Date')
    axes[1, 1].set_ylabel('Trend')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

# Fit Prophet model
if len(df_with_features) > 0:
    prophet_result = fit_prophet_model(df_with_features)
    
    if prophet_result:
        # Plot Prophet results
        plot_prophet_results(prophet_result)
        
        # Future forecasting
        print("\n🔮 Creating future forecasts...")
        
        # Create future dates (next 30 days)
        future_30_days = prophet_result['model'].make_future_dataframe(periods=30)
        future_forecast = prophet_result['model'].predict(future_30_days)
        
        # Plot future forecast
        plt.figure(figsize=(14, 6))
        plt.plot(future_forecast['ds'], future_forecast['yhat'], 'b-', linewidth=2, label='Forecast')
        plt.fill_between(future_forecast['ds'], future_forecast['yhat_lower'], 
                        future_forecast['yhat_upper'], alpha=0.3, label='Confidence Interval')
        
        # Highlight future period
        future_start = prophet_result['test_data']['ds'].max()
        plt.axvline(x=future_start, color='red', linestyle='--', alpha=0.7, label='Forecast Start')
        
        plt.title('30-Day Sales Forecast', fontsize=14, fontweight='bold')
        plt.xlabel('Date')
        plt.ylabel('Predicted Sales ($)')
        plt.legend()
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Print future forecast summary
        future_period = future_forecast[future_forecast['ds'] > future_start]
        avg_daily_forecast = future_period['yhat'].mean()
        total_forecast = future_period['yhat'].sum()
        
        print(f"   📊 30-Day Forecast Summary:")
        print(f"      Average daily sales: ${avg_daily_forecast:,.2f}")
        print(f"      Total 30-day sales: ${total_forecast:,.2f}")
    
else:
    print("⚠️ No data available for Prophet modeling")
    prophet_result = None

## 7. Model Comparison and Performance Analysis

### Comprehensive Model Comparison
Comparing all models side-by-side to identify the best performing approach.

In [ ]:
def create_model_comparison(ml_results, arima_sarima_results, prophet_result):
    """
    Create comprehensive comparison of all models.
    """
    print("\n📊 COMPREHENSIVE MODEL COMPARISON")
    print("=" * 60)
    
    comparison_data = []
    
    # Add ML models
    for model_name, results in ml_results.items():
        comparison_data.append({
            'Model': model_name,
            'Type': 'Machine Learning',
            'RMSE': results['rmse'],
            'MAE': results['mae'],
            'R²': results.get('r2', 'N/A'),
            'MAPE': results.get('mape', 'N/A')
        })
    
    # Add ARIMA/SARIMA models
    for model_name, results in arima_sarima_results.items():
        comparison_data.append({
            'Model': model_name,
            'Type': 'Time Series',
            'RMSE': results['rmse'],
            'MAE': results['mae'],
            'R²': 'N/A',
            'MAPE': 'N/A'
        })
    
    # Add Prophet model
    if prophet_result:
        comparison_data.append({
            'Model': 'Prophet',
            'Type': 'Time Series',
            'RMSE': prophet_result['rmse'],
            'MAE': prophet_result['mae'],
            'R²': prophet_result['r2'],
            'MAPE': 'N/A'
        })
    
    if comparison_data:
        # Create comparison DataFrame
        comparison_df = pd.DataFrame(comparison_data)
        
        # Display comparison table
        print("\n📋 Model Performance Summary:")
        display(comparison_df.round(3))
        
        # Find best model by RMSE
        best_model_idx = comparison_df['RMSE'].idxmin()
        best_model = comparison_df.loc[best_model_idx]
        
        print(f"\n🏆 BEST PERFORMING MODEL")
        print(f"   Model: {best_model['Model']}")
        print(f"   Type: {best_model['Type']}")
        print(f"   RMSE: ${best_model['RMSE']:,.2f}")
        print(f"   MAE: ${best_model['MAE']:,.2f}")
        
        # Create visualization
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        # RMSE comparison
        colors = ['lightblue' if t == 'Machine Learning' else 'lightcoral' 
                 for t in comparison_df['Type']]
        axes[0].bar(comparison_df['Model'], comparison_df['RMSE'], color=colors)
        axes[0].set_title('Model Comparison - RMSE', fontweight='bold')
        axes[0].set_ylabel('RMSE ($)')
        axes[0].tick_params(axis='x', rotation=45)
        
        # MAE comparison
        axes[1].bar(comparison_df['Model'], comparison_df['MAE'], color=colors)
        axes[1].set_title('Model Comparison - MAE', fontweight='bold')
        axes[1].set_ylabel('MAE ($)')
        axes[1].tick_params(axis='x', rotation=45)
        
        # Add legend
        from matplotlib.patches import Patch
        legend_elements = [Patch(facecolor='lightblue', label='Machine Learning'),
                          Patch(facecolor='lightcoral', label='Time Series')]
        axes[0].legend(handles=legend_elements)
        axes[1].legend(handles=legend_elements)
        
        plt.tight_layout()
        plt.show()
        
        return comparison_df, best_model
    else:
        print("   ⚠️ No models available for comparison")
        return None, None

# Create model comparison
comparison_df, best_model = create_model_comparison(ml_results, arima_sarima_results, prophet_result)

# Model insights and recommendations
if comparison_df is not None:
    print("\n💡 MODEL INSIGHTS & RECOMMENDATIONS")
    print("=" * 50)
    
    # Analyze model types performance
    ml_models = comparison_df[comparison_df['Type'] == 'Machine Learning']
    ts_models = comparison_df[comparison_df['Type'] == 'Time Series']
    
    if len(ml_models) > 0 and len(ts_models) > 0:
        ml_avg_rmse = ml_models['RMSE'].mean()
        ts_avg_rmse = ts_models['RMSE'].mean()
        
        print(f"Average RMSE - ML Models: ${ml_avg_rmse:,.2f}")
        print(f"Average RMSE - Time Series Models: ${ts_avg_rmse:,.2f}")
        
        if ml_avg_rmse < ts_avg_rmse:
            print("\n🎯 Machine Learning models generally outperform time series models")
            print("   This suggests strong feature relationships beyond temporal patterns")
        else:
            print("\n🎯 Time series models generally outperform machine learning models")
            print("   This suggests strong temporal dependencies in the data")
    
    # Specific recommendations
    print("\n📋 RECOMMENDATIONS:")
    if best_model['Type'] == 'Machine Learning':
        print("   • Use machine learning approach for short-term predictions")
        print("   • Consider ensemble methods combining ML and time series")
        print("   • Focus on feature engineering and external factors")
    else:
        print("   • Use time series approach for medium to long-term forecasting")
        print("   • Consider Prophet for automatic seasonality detection")
        print("   • Monitor for structural breaks in time series")
    
    print("   • Implement ensemble forecasting for robust predictions")
    print("   • Regular model retraining and validation")
    print("   • Monitor forecast accuracy in production")

## 8. Interactive Visualizations with Plotly

### Enhanced Interactive Dashboard
Creating interactive visualizations for better insights and presentation.

In [ ]:
def create_interactive_sales_dashboard(df):
    """
    Create interactive sales dashboard using Plotly.
    """
    print("🎨 Creating interactive sales dashboard...")
    
    # Prepare daily sales data
    daily_sales = df.groupby('date').agg({
        'total_sales': 'sum',
        'item_count': 'sum',
        'store_id': 'nunique'
    }).reset_index()
    
    daily_sales['avg_transaction'] = daily_sales['total_sales'] / daily_sales['item_count']
    daily_sales['rolling_7d'] = daily_sales['total_sales'].rolling(window=7).mean()
    
    # Create subplot dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Daily Sales Trend', 'Sales by Category', 
                       'Restaurant Performance', 'Monthly Sales Pattern'),
        specs=[[{"secondary_y": True}, {"type": "pie"}],
               [{"type": "bar"}, {"type": "box"}]]
    )
    
    # 1. Daily sales trend with rolling average
    fig.add_trace(
        go.Scatter(x=daily_sales['date'], y=daily_sales['total_sales'],
                  mode='lines', name='Daily Sales', opacity=0.6,
                  line=dict(color='lightblue')),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=daily_sales['date'], y=daily_sales['rolling_7d'],
                  mode='lines', name='7-Day Average',
                  line=dict(color='red', width=2)),
        row=1, col=1
    )
    
    # 2. Sales by category (pie chart)
    category_sales = df.groupby('category')['total_sales'].sum().reset_index()
    fig.add_trace(
        go.Pie(labels=category_sales['category'], values=category_sales['total_sales'],
               name="Category Sales"),
        row=1, col=2
    )
    
    # 3. Restaurant performance (bar chart)
    restaurant_sales = df.groupby('restaurant_name')['total_sales'].sum().reset_index()
    restaurant_sales = restaurant_sales.sort_values('total_sales', ascending=True)
    
    fig.add_trace(
        go.Bar(x=restaurant_sales['total_sales'], y=restaurant_sales['restaurant_name'],
               orientation='h', name="Restaurant Sales",
               marker_color='lightgreen'),
        row=2, col=1
    )
    
    # 4. Monthly sales distribution (box plot)
    df_monthly = df.copy()
    df_monthly['month_name'] = df_monthly['date'].dt.strftime('%B')
    month_order = ['January', 'February', 'March', 'April', 'May', 'June',
                  'July', 'August', 'September', 'October', 'November', 'December']
    
    for month in month_order:
        if month in df_monthly['month_name'].values:
            month_data = df_monthly[df_monthly['month_name'] == month]['total_sales']
            fig.add_trace(
                go.Box(y=month_data, name=month[:3], showlegend=False),
                row=2, col=2
            )
    
    # Update layout
    fig.update_layout(
        title_text="Interactive Sales Analytics Dashboard",
        title_x=0.5,
        title_font_size=16,
        height=800,
        showlegend=True
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_yaxes(title_text="Sales ($)", row=1, col=1)
    fig.update_xaxes(title_text="Sales ($)", row=2, col=1)
    fig.update_yaxes(title_text="Restaurant", row=2, col=1)
    fig.update_yaxes(title_text="Sales ($)", row=2, col=2)
    
    fig.show()
    
    return fig

def create_forecast_comparison_plot(comparison_df, ml_results, prophet_result):
    """
    Create interactive forecast comparison visualization.
    """
    if comparison_df is None:
        print("⚠️ No comparison data available for visualization")
        return
    
    print("🎨 Creating interactive forecast comparison...")
    
    # Create interactive model performance comparison
    fig = go.Figure()
    
    # Add RMSE bars
    colors = ['lightblue' if t == 'Machine Learning' else 'lightcoral' 
             for t in comparison_df['Type']]
    
    fig.add_trace(go.Bar(
        x=comparison_df['Model'],
        y=comparison_df['RMSE'],
        name='RMSE',
        marker_color=colors,
        text=[f'${x:,.0f}' for x in comparison_df['RMSE']],
        textposition='auto',
        hovertemplate='<b>%{x}</b><br>RMSE: $%{y:,.0f}<extra></extra>'
    ))
    
    fig.update_layout(
        title='Interactive Model Performance Comparison',
        title_x=0.5,
        xaxis_title='Model',
        yaxis_title='RMSE ($)',
        hovermode='x unified',
        height=500
    )
    
    fig.show()
    
    # If Prophet results available, create forecast visualization
    if prophet_result:
        forecast = prophet_result['forecast']
        train_data = prophet_result['train_data']
        test_data = prophet_result['test_data']
        
        fig_forecast = go.Figure()
        
        # Add training data
        fig_forecast.add_trace(go.Scatter(
            x=train_data['ds'],
            y=train_data['y'],
            mode='lines',
            name='Training Data',
            line=dict(color='blue', width=2)
        ))
        
        # Add test data
        fig_forecast.add_trace(go.Scatter(
            x=test_data['ds'],
            y=test_data['y'],
            mode='lines',
            name='Actual Test Data',
            line=dict(color='green', width=2)
        ))
        
        # Add forecast
        fig_forecast.add_trace(go.Scatter(
            x=forecast['ds'],
            y=forecast['yhat'],
            mode='lines',
            name='Prophet Forecast',
            line=dict(color='red', width=2, dash='dash')
        ))
        
        # Add confidence interval
        fig_forecast.add_trace(go.Scatter(
            x=forecast['ds'],
            y=forecast['yhat_upper'],
            mode='lines',
            line=dict(width=0),
            showlegend=False,
            hoverinfo='skip'
        ))
        
        fig_forecast.add_trace(go.Scatter(
            x=forecast['ds'],
            y=forecast['yhat_lower'],
            mode='lines',
            line=dict(width=0),
            fillcolor='rgba(255,0,0,0.2)',
            fill='tonexty',
            name='Confidence Interval',
            hoverinfo='skip'
        ))
        
        fig_forecast.update_layout(
            title='Interactive Sales Forecast (Prophet Model)',
            title_x=0.5,
            xaxis_title='Date',
            yaxis_title='Sales ($)',
            hovermode='x unified',
            height=500
        )
        
        fig_forecast.show()

# Create interactive visualizations
if len(df_with_features) > 0:
    # Main dashboard
    dashboard_fig = create_interactive_sales_dashboard(df_with_features)
    
    # Forecast comparison
    create_forecast_comparison_plot(comparison_df, ml_results, prophet_result)
else:
    print("⚠️ No data available for interactive visualizations")

## 9. Business Insights and Conclusions

### Key Findings and Actionable Insights
Summarizing the analysis results and providing business recommendations.

In [ ]:
def generate_business_insights(df, comparison_df, best_model):
    """
    Generate comprehensive business insights from the analysis.
    """
    print("\n" + "=" * 70)
    print("🏢 BUSINESS INSIGHTS & STRATEGIC RECOMMENDATIONS")
    print("=" * 70)
    
    # Sales Performance Insights
    print("\n📊 SALES PERFORMANCE ANALYSIS")
    print("-" * 40)
    
    total_sales = df['total_sales'].sum()
    avg_daily_sales = df.groupby('date')['total_sales'].sum().mean()
    total_transactions = len(df)
    avg_transaction_value = df['total_sales'].mean()
    
    print(f"💰 Total Sales Revenue: ${total_sales:,.2f}")
    print(f"📅 Average Daily Sales: ${avg_daily_sales:,.2f}")
    print(f"🛒 Total Transactions: {total_transactions:,}")
    print(f"💳 Average Transaction Value: ${avg_transaction_value:.2f}")
    
    # Temporal Insights
    print("\n⏰ TEMPORAL PATTERNS")
    print("-" * 30)
    
    # Day of week analysis
    dow_sales = df.groupby(df['date'].dt.day_name())['total_sales'].sum()
    best_day = dow_sales.idxmax()
    worst_day = dow_sales.idxmin()
    
    print(f"🏆 Best performing day: {best_day} (${dow_sales[best_day]:,.2f})")
    print(f"📉 Lowest performing day: {worst_day} (${dow_sales[worst_day]:,.2f})")
    
    # Weekend vs weekday
    df_temp = df.copy()
    df_temp['is_weekend'] = df_temp['date'].dt.dayofweek.isin([5, 6])
    weekend_avg = df_temp[df_temp['is_weekend']]['total_sales'].mean()
    weekday_avg = df_temp[~df_temp['is_weekend']]['total_sales'].mean()
    weekend_premium = ((weekend_avg / weekday_avg) - 1) * 100
    
    print(f"🎉 Weekend vs Weekday Premium: {weekend_premium:+.1f}%")
    
    # Category Performance
    print("\n🍽️ CATEGORY PERFORMANCE")
    print("-" * 30)
    
    category_performance = df.groupby('category').agg({
        'total_sales': ['sum', 'mean'],
        'item_count': 'sum'
    }).round(2)
    
    category_performance.columns = ['Total_Sales', 'Avg_Transaction', 'Total_Items']
    category_performance = category_performance.sort_values('Total_Sales', ascending=False)
    
    print("Top performing categories:")
    for i, (category, row) in enumerate(category_performance.head(3).iterrows()):
        print(f"  {i+1}. {category}: ${row['Total_Sales']:,.2f} total, ${row['Avg_Transaction']:.2f} avg")
    
    # Restaurant Performance
    print("\n🏪 RESTAURANT PERFORMANCE")
    print("-" * 30)
    
    restaurant_performance = df.groupby('restaurant_name').agg({
        'total_sales': ['sum', 'mean', 'count'],
        'item_count': 'sum'
    }).round(2)
    
    restaurant_performance.columns = ['Total_Sales', 'Avg_Transaction', 'Num_Transactions', 'Total_Items']
    restaurant_performance = restaurant_performance.sort_values('Total_Sales', ascending=False)
    
    print("Top performing restaurants:")
    for i, (restaurant, row) in enumerate(restaurant_performance.head(3).iterrows()):
        print(f"  {i+1}. {restaurant}: ${row['Total_Sales']:,.2f} total, {row['Num_Transactions']} transactions")
    
    # Model Performance Insights
    if comparison_df is not None and best_model is not None:
        print("\n🤖 FORECASTING MODEL INSIGHTS")
        print("-" * 35)
        
        print(f"🏆 Best Model: {best_model['Model']} ({best_model['Type']})")
        print(f"📊 Forecast Accuracy: RMSE ${best_model['RMSE']:,.2f}")
        
        # Model type analysis
        ml_models = comparison_df[comparison_df['Type'] == 'Machine Learning']
        ts_models = comparison_df[comparison_df['Type'] == 'Time Series']
        
        if len(ml_models) > 0 and len(ts_models) > 0:
            ml_avg_rmse = ml_models['RMSE'].mean()
            ts_avg_rmse = ts_models['RMSE'].mean()
            
            if ml_avg_rmse < ts_avg_rmse:
                better_approach = "Machine Learning"
                performance_diff = ((ts_avg_rmse / ml_avg_rmse) - 1) * 100
            else:
                better_approach = "Time Series"
                performance_diff = ((ml_avg_rmse / ts_avg_rmse) - 1) * 100
            
            print(f"📈 {better_approach} models perform {performance_diff:.1f}% better on average")
    
    # Strategic Recommendations
    print("\n" + "=" * 70)
    print("🎯 STRATEGIC RECOMMENDATIONS")
    print("=" * 70)
    
    print("\n💡 OPERATIONAL RECOMMENDATIONS:")
    print(f"   • Focus marketing efforts on {best_day}s for maximum impact")
    if weekend_premium > 0:
        print(f"   • Leverage weekend premium ({weekend_premium:.1f}%) with special promotions")
    else:
        print(f"   • Boost weekend sales with targeted promotions (currently {weekend_premium:.1f}% below weekdays)")
    
    top_category = category_performance.index[0]
    print(f"   • Expand {top_category} offerings as the top-performing category")
    
    top_restaurant = restaurant_performance.index[0]
    print(f"   • Replicate {top_restaurant}'s success strategies across other locations")
    
    print("\n📊 FORECASTING RECOMMENDATIONS:")
    if best_model is not None:
        if best_model['Type'] == 'Machine Learning':
            print("   • Implement ML-based forecasting for short-term planning")
            print("   • Focus on feature engineering and external data integration")
            print("   • Monitor key predictive features for early trend detection")
        else:
            print("   • Use time series models for medium to long-term planning")
            print("   • Monitor seasonal patterns for inventory planning")
            print("   • Consider external factors affecting time series patterns")
    
    print("   • Implement ensemble forecasting for robust predictions")
    print("   • Set up automated model retraining pipelines")
    print("   • Establish forecast accuracy monitoring and alerting")
    
    print("\n🚀 GROWTH OPPORTUNITIES:")
    print("   • Optimize menu mix based on category performance analysis")
    print("   • Implement dynamic pricing during peak periods")
    print("   • Develop targeted customer acquisition strategies")
    print("   • Consider expansion in high-performing restaurant locations")
    print("   • Implement predictive inventory management")
    
    print("\n📈 NEXT STEPS:")
    print("   1. Deploy selected forecasting model to production")
    print("   2. Set up real-time data collection and monitoring")
    print("   3. Implement A/B testing for recommended strategies")
    print("   4. Establish KPIs and success metrics")
    print("   5. Schedule regular model performance reviews")
    
    return {
        'total_sales': total_sales,
        'avg_daily_sales': avg_daily_sales,
        'best_day': best_day,
        'weekend_premium': weekend_premium,
        'top_category': top_category,
        'top_restaurant': top_restaurant
    }

# Generate comprehensive business insights
if len(df_with_features) > 0:
    business_insights = generate_business_insights(df_with_features, comparison_df, best_model)
else:
    print("⚠️ No data available for business insights generation")
    
print("\n" + "=" * 70)
print("✅ ANALYSIS COMPLETE")
print("=" * 70)
print("\nThis comprehensive sales forecasting analysis provides:")
print("• Advanced time series forecasting capabilities")
print("• Multiple model comparison and validation")
print("• Interactive visualizations for stakeholder presentation")
print("• Actionable business insights and recommendations")
print("• Portable code that works across different environments")
print("\nThe notebook is now ready for deployment and further customization!")